# Stepper dancer

In [4]:
import serial
import time
import ctypes
import perlin_noise
import datetime
import random

import pyaudio
import math

# Create an instance of PyAudio
audio = pyaudio.PyAudio()


In [45]:
# Set up serial communications
port = "/dev/cu.usbserial-1420"
baud = 115200
ser = serial.Serial(port, baud)
time.sleep(2)

In [32]:
# Open a stream to capture audio from the microphone
stream = audio.open(format=pyaudio.paInt16,
                    channels=1,
                    rate=44100,
                    input=True,
                    frames_per_buffer=1024)

In [34]:
def serialize(motor, acceleration, speed, revolutions):
    motor_b = int(motor).to_bytes()
    acceleration_b = acceleration.to_bytes(2, signed = True)
    speed_b = speed.to_bytes(2, signed = True)
    revolutions_b = int(revolutions * 16.0 * 200.0).to_bytes(2, signed = True)
    print(( motor_b + acceleration_b + speed_b + revolutions_b).hex(' ', 2) )
    return motor_b + acceleration_b + speed_b + revolutions_b

def serialize_float(motor, acceleration, speed, revolutions):
    acceleration_32 = ctypes.c_float(acceleration)
    speed_32 = ctypes.c_float(speed)
    revolutions_32 = ctypes.c_float(revolutions)
    
    motor_b = int(motor).to_bytes()
    acceleration_b = bytes(acceleration_32)
    speed_b = bytes(speed_32)
    revolutions_b = bytes(revolutions_32)
    return (motor_b + acceleration_b + speed_b + revolutions_b)

def get_rms(frames):
    count = len(frames)/2.0
    sum_squares = 0.0
    sum_samples = 0.0

    for i in range(0, len(frames), 2):
        # Unpack the frame (2 bytes) into a signed integer
        n = int.from_bytes(frames[i:i+2], byteorder='little', signed=True)
        sum_squares += n ** 2
        sum_samples += n

    rms = math.sqrt(sum_squares / count)
    dc = sum_samples / count
    return rms
    return (rms, dc)

In [46]:
acceleration = 1000.0
moving_average = 0.0
moving_average_num = 5.0

while True:
    # Read audio data from the stream
    frames = b''
    for i in range(4):
        data = stream.read(1024, exception_on_overflow = False)
        frames += data

    # print("Finished recording.")
    rms_value = get_rms(frames)
    moving_average = (moving_average * moving_average_num + rms_value) / (moving_average_num + 1)
    # print(f"values: {rms_value}")

    myspeed = moving_average / 5000.0
    print(myspeed)
    ser.write( serialize_float(motor = 0, acceleration = acceleration, speed = myspeed, revolutions = 1 ) )
    ser.write( serialize_float(motor = 1, acceleration = acceleration, speed = myspeed, revolutions = 1 ) )
    ser.write( serialize_float(motor = 2, acceleration = acceleration, speed = myspeed, revolutions = 1 ) )
    ser.write( serialize_float(motor = 3, acceleration = acceleration, speed = myspeed, revolutions = 1 ) )
    # ser.write( serialize_float(motor = 1, acceleration = acceleration, speed = myspeed, revolutions = 0.1  ) )


0.0003864871114621921
0.0031845674755225078
0.004269079351744157
0.004045562501685477
0.0052195057893099205
0.005279202271341753
0.004887094182892059
0.0046448539318337395
0.004268981590985541
0.004264399407459905
0.0039909813620235095
0.00387951213948001
0.004076256153361712
0.004060819250327083
0.0038900803404416594
0.003678400730981688
0.0034468112056915395
0.003376509338076283
0.003264831924133061
0.0031631214903169672
0.0029483356012574395
0.002787911637221282
0.002690999382962551
0.0025909634546522863
0.002751601699587047
0.0026971268034971484
0.002651752536041825
0.0025346042549553157
0.002455773795548021
0.002441983098002801
0.003025309514217861
0.012758846322858229
0.031171122072017506
0.030057314535160078
0.02646253432173486
0.0257270122841933
0.023441371405372467
0.02360553373117871
0.02456013520125074
0.0227339625269973
0.02303807364373904
0.02224880651253296
0.01915303810626226
0.016565646340256326
0.014336838689162041
0.01249507921697915
0.010932082578791363
0.00965650365

KeyboardInterrupt: 

In [6]:
stream.stop_stream()
stream.close()
audio.terminate()